In [16]:
import lpips
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

import torch
img0 = torch.zeros(1,3,64,64) # image should be RGB, IMPORTANT: normalized to [-1,1]
img1 = torch.zeros(1,3,64,64)
d = loss_fn_alex(img0, img1)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/minha/dev/tinova/ai-copyright-portection/.venv/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /Users/minha/dev/tinova/ai-copyright-portection/.venv/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


In [17]:
from PIL import Image
import torchvision.transforms as transforms

# 이미지 로드 및 전처리
def load_image(path):
    image = Image.open(path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)  # LPIPS는 [-1, 1] 입력 필요
    ])
    return transform(image).unsqueeze(0)

img_path1 = load_image("../data/images/dataset/ex1.jpeg")
img_path2 = load_image("../data/images/dataset/ex2.jpeg")

# 유사도 계산
with torch.no_grad():
    dist = loss_fn_alex(img_path1, img_path2)

print("LPIPS distance:", dist.item())  # 0에 가까울수록 유사함

LPIPS distance: 0.373117595911026


| LPIPS 점수     | 유사도 평가 | 비고                          |
| ------------ | ------ | --------------------------- |
| 0.00 \~ 0.20 | 매우 유사  | 구도, 색감, 질감까지 거의 동일한 경우 많음   |
| 0.20 \~ 0.40 | 부분 유사  | 구성이나 스타일 유사하지만 세부 디테일 차이 있음 |
| 0.40 \~ 0.70 | 낮은 유사도 | 대체로 다른 그림으로 간주됨             |
| 0.70 \~ 1.00 | 거의 무관  | 전혀 다른 이미지                   |

### Style Loss 손실 함수 추가

In [34]:
# 디바이스 설정
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# LPIPS 모델 로딩
lpips_model = lpips.LPIPS(net='alex')
lpips_model = lpips_model.to(device)  # 여기까지는 잘 하고 있음

# 내부 ScalingLayer 문제 해결을 위해 모델 내부도 옮겨야 함
lpips_model.scaling_layer.shift = lpips_model.scaling_layer.shift.to(device)
lpips_model.scaling_layer.scale = lpips_model.scaling_layer.scale.to(device)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/minha/dev/tinova/ai-copyright-portection/.venv/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [35]:
# Gram Matrix 계산

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(b, c, h * w)
    G = torch.bmm(features, features.transpose(1, 2))
    return G / (c * h * w)

In [36]:
# StyleLoss 클래스
import torch.nn as nn
import torchvision.models as models

device = 'mps'

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        loss = nn.functional.mse_loss(G, self.target)
        return loss

In [37]:
# VGG에서 style feature 추출
def get_style_feature(image, layer_idx=0):    
    vgg = models.vgg19(pretrained=True).features.to(device).eval()
    for i, layer in enumerate(vgg):
        image = layer(image)
        if i == layer_idx:
            return image
    raise ValueError("Layer index too high")

In [40]:
# 유사도 계산
def compute_similarity(img_path1, img_path2, alpha=0.5, beta=0.5):
    img1 = load_image(img_path1).to(device)
    img2 = load_image(img_path2).to(device)

    # LPIPS
    lpips_model = lpips.LPIPS(net='alex').to(device)
    lpips_score = lpips_model(img1, img2).item()

    # Style Loss
    feat1 = get_style_feature(img1, layer_idx=10)  # conv3_1
    feat2 = get_style_feature(img2, layer_idx=10)
    style_loss_fn = StyleLoss(feat1)
    style_score = style_loss_fn(feat2).item()

    # Normalize style score (선택적)
    style_score = min(style_score, 1.0)

    # 조합 점수 (가중치 합산)
    final_score = alpha * lpips_score + beta * style_score

    return {
        "lpips_score": lpips_score,
        "style_score": style_score,
        "combined_score": final_score
    }

In [41]:
img_path1 = load_image("../data/images/dataset/ex1.jpeg")
img_path2 = load_image("../data/images/dataset/ex2.jpeg")

result = compute_similarity("../data/images/dataset/ex1.jpeg", "../data/images/dataset/ex2.jpeg")

print("LPIPS:", result["lpips_score"])
print("Style Loss:", result["style_score"])
print("Combined Score:", result["combined_score"])

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/minha/dev/tinova/ai-copyright-portection/.venv/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /Users/minha/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


100%|██████████| 548M/548M [00:47<00:00, 12.0MB/s] 


LPIPS: 0.3731175661087036
Style Loss: 6.617650797124952e-05
Combined Score: 0.18659187130833743
